In [1]:
##导入必要的库
# coding=UTF-8
import xlrd
import xlwt
from xlutils.copy import copy
import requests
import time
import hashlib
import json

In [2]:
##用于读取企业名单
def load_data(path):
    workbook = xlrd.open_workbook(path)  # 打开工作簿
    sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
    worksheet = workbook.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
    ####测试仅仅使用名单前3家###
    for i in range(1, 4):
        print(worksheet.cell_value(i, 1), "\t", end="")  # 逐行逐列读取数据
        print()

In [3]:
##用于写入数据
def write_excel_xls(path, sheet_name, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlwt.Workbook()  # 新建一个工作簿
    sheet = workbook.add_sheet(sheet_name)  # 在工作簿中新建一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i, j, value[i][j])  # 像表格中写入数据（对应的行和列）
    workbook.save(path)  # 保存工作簿
    print("xls格式表格写入数据成功！")

In [4]:
##用于增补数据
def write_excel_xls_append(path, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlrd.open_workbook(path)  # 打开工作簿
    sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
    worksheet = workbook.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
    rows_old = worksheet.nrows  # 获取表格中已存在的数据的行数
    new_workbook = copy(workbook)  # 将xlrd对象拷贝转化为xlwt对象
    new_worksheet = new_workbook.get_sheet(0)  # 获取转化后工作簿中的第一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            new_worksheet.write(i+rows_old, j, value[i][j])  # 追加写入数据，注意是从i+rows_old行开始写入
    new_workbook.save(path)  # 保存工作簿

In [5]:
##准备工作
#  token可以从 数据中心 -> 我的接口 中获取
token = "9320ff98-a6be-4136-8d81-49cccf76a5bc"
encode = 'utf-8'
path = '名单.xls'
book_name_xls = '数据写入.xls' 
sheet_name_xls = '抓取数据写入' 
value_title = [["企业名", "风险时间", "经营异常原因"],]
write_excel_xls(book_name_xls, sheet_name_xls, value_title)

xls格式表格写入数据成功！


# SOS 运行前请检查并确认“名单.xls”、“数据写入.xls”

In [6]:
workbook = xlrd.open_workbook(path)  # 打开工作簿
sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
worksheet = workbook.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
for i in range(1,worksheet.nrows):
    corname = worksheet.cell_value(i, 1) # 逐行逐列读取数据
    
    url = "http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword="+corname+"&pageNum=1"
    headers={'Authorization': token}
    response = requests.get(url, headers=headers)
    print(url)
    ##结果处理  
    m=json.loads(response.text)
    #print(m)
    if m["result"] == None:
        value = [[corname,"无数据"],]
        write_excel_xls_append(book_name_xls, value)
        print("%s无经营异常数据"%corname)
    else:
        total = m["result"]["total"]
        for i in range(total):
            value = [[corname,m["result"]["items"][i]["putDate"],m["result"]["items"][i]["putReason"]]]
            write_excel_xls_append(book_name_xls, value)
            print(corname+"第"+str(i+1)+"条数据写入成功")

http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=佛山市金融投资控股有限公司&pageNum=1
佛山市金融投资控股有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=七匹狼控股集团股份有限公司&pageNum=1
七匹狼控股集团股份有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=广东广物金融产业集团有限公司&pageNum=1
广东广物金融产业集团有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=江苏省信用再担保集团有限公司&pageNum=1
江苏省信用再担保集团有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=山东易通发展集团有限公司&pageNum=1
山东易通发展集团有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=广东粤财投资控股有限公司&pageNum=1
广东粤财投资控股有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=清控资产管理有限公司&pageNum=1
清控资产管理有限公司无经营异常数据
http://open.api.tianyancha.com/services/open/mr/abnormal/2.0?pageSize=20&keyword=高晟财富控股集团有限公司&pageNum=1
高晟财富控股集团有限公司无经营异常数据
